In [1]:
import pytest
import pennylane as qml
from pennylane import QubitStateVector, BasisState, DeviceError
from pennylane.devices import DefaultMixed
from pennylane.ops import (
    Identity,
    PauliZ,
    CZ,
    PauliX,
    Hadamard,
    CNOT,
    ISWAP,
    AmplitudeDamping,
    DepolarizingChannel,
    ResetError,
    PauliError,
)
from pennylane.wires import Wires

import numpy as np

In [17]:
def root_state(nr_wires):
    """Pure state with equal amplitudes but phases equal to roots of unity"""
    dim = 2**nr_wires
    ket = [np.exp(1j * 2 * np.pi * n / dim) / np.sqrt(dim) for n in range(dim)]
    return np.outer(ket, np.conj(ket))

def test(x):
    nr_wires = x[0]
    op = x[1]
    target_state = np.reshape(x[2], [2] * 2 * nr_wires)
    dev = qml.device("default.mixed", wires=nr_wires)
    root = np.reshape(root_state(nr_wires), [2] * 2 * nr_wires)
    dev._state = root
    kraus = dev._get_kraus(op)
    dev._apply_channel(kraus, wires=Wires([0, 1]))

    print(dev._state)

    return np.allclose(dev._state, target_state, atol=1e-3, rtol=0)

In [22]:
x = [2, ISWAP(wires=[0, 1]), 
    np.array(
        [
            [0.25 + 0.0j, 0.0 + 0.25j, -0.25 + 0.0, 0.0 + 0.25j],
            [0.0 - 0.25j, 0.25 + 0.0j, 0.0 + 0.25j, 0.25 + 0.0j],
            [-0.25 - 0.0j, 0.0 - 0.25j, 0.25 + 0.0j, 0.0 - 0.25j],
            [0.0 - 0.25j, 0.25 + 0.0j, 0.0 + 0.25j, 0.25 + 0.0j],
        ]
    )]

In [23]:
test(x)

[[[[ 2.5000000e-01+0.0000000e+00j -3.0616170e-17+2.5000000e-01j]
   [-2.5000000e-01-1.5308085e-17j -4.5924255e-17+2.5000000e-01j]]

  [[-3.0616170e-17-2.5000000e-01j  2.5000000e-01+0.0000000e+00j]
   [ 1.5308085e-17+2.5000000e-01j  2.5000000e-01+1.5308085e-17j]]]


 [[[-2.5000000e-01+1.5308085e-17j  1.5308085e-17-2.5000000e-01j]
   [ 2.5000000e-01+0.0000000e+00j  3.0616170e-17-2.5000000e-01j]]

  [[-4.5924255e-17-2.5000000e-01j  2.5000000e-01-1.5308085e-17j]
   [ 3.0616170e-17+2.5000000e-01j  2.5000000e-01+0.0000000e+00j]]]]


array([[[[ 0.0000000e+00+0.0000000e+00j, -3.0616170e-17+0.0000000e+00j],
         [ 0.0000000e+00-1.5308085e-17j, -4.5924255e-17+0.0000000e+00j]],

        [[-3.0616170e-17+0.0000000e+00j,  0.0000000e+00+0.0000000e+00j],
         [ 1.5308085e-17+0.0000000e+00j,  0.0000000e+00+1.5308085e-17j]]],


       [[[ 0.0000000e+00+1.5308085e-17j,  1.5308085e-17+0.0000000e+00j],
         [ 0.0000000e+00+0.0000000e+00j,  3.0616170e-17+0.0000000e+00j]],

        [[-4.5924255e-17+0.0000000e+00j,  0.0000000e+00-1.5308085e-17j],
         [ 3.0616170e-17+0.0000000e+00j,  0.0000000e+00+0.0000000e+00j]]]])